In [14]:
import numpy as np
import pandas as pd
import librosa as lr
import glob
import os
from pyroomacoustics.experimental.rt60 import measure_rt60

from IPython.display import Audio

In [15]:
rir_df = pd.read_csv('/home/cxw/volest/volume-estimation/data/0812_rir_df.csv')

In [16]:
sr = 16000
rir_sample, _ = lr.load(rir_df.iloc[200]['file'], sr=sr)

In [17]:
Audio(data=rir_sample, rate=sr)

In [18]:
rt60_sample = measure_rt60(rir_sample, fs=sr, decay_db=60)

In [19]:
rt60_sample

13.0951875

In [20]:
def compute_rt60(rir_path, sr=16000):
    rir, _ = lr.load(rir_path, sr=sr)
    rt60 = measure_rt60(rir, fs=sr)
    return rt60

In [21]:
compute_rt60(rir_df.iloc[200]['file'])

13.0951875

In [22]:
files = rir_df['file'].tolist()
rt60_dict = {}
bad_files = []
for file in files:
    try:
        rt60 = compute_rt60(file)
    except:
        print(file)
        bad_files.append(file)
        continue
    rt60_dict[file] = rt60

/home/cxw/sonos_rirs/rirs/openair/ron-cooke-hub-university-york/b-format/sssr.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID03_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID06_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID05_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID11_20180110_T/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v01.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID04_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID08_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID09_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav


In [23]:
rt60_list = []
for row in rir_df.iterrows():
    file = row[1]['file']
    try:
        rt60 = compute_rt60(file)
    except:
        print(file)
        rt60 = None
        pass
    rt60_list.append(rt60)
rir_df['computed_rt60'] = rt60_list

/home/cxw/sonos_rirs/rirs/openair/ron-cooke-hub-university-york/b-format/sssr.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID03_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID06_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID05_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID11_20180110_T/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v01.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID04_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID08_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav
/home/cxw/sonos_rirs/rirs/BUT_ReverbDB/VUT_FIT_L227/MicID01/SpkID09_20180105_S/01/RIR/IR_sweep_15s_45Hzto22kHz_FS16kHz.v00.wav


In [24]:
rooms = rir_df['room'].unique()
gb_room = rir_df.groupby('room')
mean_rt60_dict = {}
for room in rooms:
    mean_rt60 = gb_room.get_group(room)['computed_rt60'].mean()
    mean_rt60_dict[room] = mean_rt60

In [25]:
rir_df['mean_rt60'] = rir_df['room'].apply(lambda row:mean_rt60_dict[row])
rir_df.to_csv('/home/cxw/volest/volume-estimation/data/0812_rir_df.csv')

In [26]:
features_folder = '/home/cxw/sonos_rirs/features/081522_20k'
experiments_081822 = glob.glob(os.path.join(features_folder, "081822_*"))
experiments_081822.sort()
room_rir_df = rir_df[['room', 'mean_rt60']]

for exp in experiments_081822:
    csv_path = os.path.join(features_folder, exp, 'feature_df.csv')
    exp_df = pd.read_csv(csv_path)
    print(exp_df.columns)
    exp_df = exp_df.merge(room_rir_df, on=['room'], how='left')
    exp_df = exp_df.rename(mapper={"mean_rt60":"rt60"}, axis='columns')
    exp_df.to_csv(csv_path, index=False)

In [29]:
AUDIO_OUT = '/home/cxw/sonos_rirs/reverb_audio'
# FEATURES_OUT = '/home/cxw/sonos_rirs/features/081522_20k'

room_rir_df = rir_df[['room', 'mean_rt60']]
#route to audio csv and load
audio_name = '080322_10K'
audio_path = os.path.join(AUDIO_OUT, audio_name)
audio_df = pd.read_csv(os.path.join(audio_path, 'audio_df.csv'))

In [30]:
audio_df = audio_df.merge(room_rir_df, on='room', how='left')\
                  .rename(mapper={"mean_rt60":"rt60"}, axis='columns')

audio_df.to_csv(os.path.join(audio_path, 'audio_df.csv'), index=False)

In [31]:
audio_df = pd.read_csv(os.path.join(audio_path, 'audio_df.csv'))
audio_df.columns

Index(['file_speech', 'length (s)', 'file_rir', 'room', 'dataset', 'vol',
       'snr', 'clip_label', 'file_clip', 'rt60'],
      dtype='object')